This notebook implements a pre-trained Inception V3 model for the task of automatic classification of breast cancer metastases in whole-slide images (WSI) of histological lymph node sections. This notebook is a simiplified version of the method proposed recently by Liu, Yun, et al. "Detecting cancer metastases on gigapixel pathology images." arXiv preprint arXiv:1703.02442 (2017). https://arxiv.org/abs/1703.02442

The image dataset was used as part of and can be obtained from Camelyon 16 (expired) or Camelyon 17 (on-going as of Sept 5, 2017) competition. Reference https://camelyon17.grand-challenge.org/


Software stacks required are:

    Python 2.7
    Keras 2.0.7
    Tensorflow 1.3.0
    Openslide 3.4.1
    Openslide Python interface 1.1.1

In [ ]:
# Let's Make the Display width of each cell the width of the window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

We visualize a few normal and tumor patch examples below. The top row is examples of tumor patches. The bottom row is example of normal patches.

*Note: We did not threshold any pixel values during the preprocessing/data curation step, therefore patches of empty spaces (all-white/grey as seen in the lower left corner subplot) pixels are present in our training/test set. One could conceivably threshold these patches out based on pixel values and simply predict them as non-tumor to save computation time.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['figure.figsize'] = (10.0, 10.0) # enlarge images

plt.figure(1)
plt.subplot(2,2,1)
img1 = np.load('/media/data/home/lizx/split/Tumor_Slide_070/Tumor/Tumor_Slide_070_Tumor_Patch_00100.npy')
plt.title('Tumor patch from tumor slide')
plt.imshow(img1)
plt.subplot(2,2,2)
img2 = np.load('/media/data/home/lizx/split/Tumor_Slide_044/Tumor/Tumor_Slide_044_Tumor_Patch_01038.npy')
plt.title('Tumor patch from tumor slide')
plt.imshow(img2)
plt.subplot(2,2,3)
img3 = np.load('/media/data/home/lizx/split/Normal_Slide_010/Normal/Normal_Slide_010_Normal_Patch_05000.npy')
plt.title('Normal patch from Normal slide')
plt.imshow(img3)
plt.subplot(2,2,4)
img3 = np.load('/media/data/home/lizx/split/Tumor_Slide_011/Normal/Tumor_Slide_011_Normal_Patch_02500.npy')
plt.title('Normal patch from tumor slide')
plt.imshow(img3)
plt.show()


In the next cell, we load in the pre-trained 48-layers Inception V3 model using Keras API. The original paper introducing the Inception V3 can be found here: https://arxiv.org/abs/1512.00567
The top layer was not included as we add global pooling layer and two fully-connected (dense) layers to modify the architecture to fit our particular dataset need.

For a friendly tutorial on Inception modules and earlier version of Inception V1 (i.e. GoogLeNet), see https://hacktilldawn.com/2016/09/25/inception-modules-explained-and-implemented/

The ImageNet dataset used to pretrain the Inception V3 model is a 1M image dataset spanning 1000 object classes as part of Imagenet Large Scale Visual recognition Challenge 2012 (ILSVRC2012). You can reference more here: http://image-net.org/challenges/LSVRC/2012/index

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.models import load_model
import os

# The tensorflow backend automatically pre-allocates all avaiable GPU resources/memory to your run. Therefore we
# explicitly limit what GPU id is avialable to be used with the following command.
os.environ["CUDA_VISIBLE_DEVICES"]="0" # set python to use only the GPU with id = 0. Can be set to "0,2" if want to use GPUs id 0 and 2.

# Create the base pre-trained model. As we are dealing with a binary classification problem instead of 1000-class problem, 
# we do not include the top layer.
base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output
# add a global spatial average pooling layer
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a sigmoid (binary) layer
predictions = Dense(1, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)



Let's print out a summary of our model and a few summary statistics. As the number of trainable parameters are large (tens of millions), this gives a good indication of how long training the entire architecture would take. 

In [ ]:
print(model.summary())

Since output shape is trimmed off due to character limits of the .summary() function, you can explictly print output shape layer by layer to see the dimensions that were cut off.

In [ ]:
# Explicit printout of layer's shape
for layer in model.layers:
    print(layer.output)

As each WSI is too large (100,000x100,000 pixels at about ~ 2GB) to be loaded in memory, we need to write a generator to return image patches of size 299x299 pixel.

299x299 is the standard input size required by Inception V3. It is also possible to change the input layer to accept an input image of a different size. 

Keras fit_generator() functions are not threadsafe, therefore we first create a theadsafe generator. See reference here: http://anandology.com/blog/using-iterators-and-generators/

We use a 80/20 training/validation data split of the training data. As the dataset is heavily imbalanced with the vast majority of the training patches belonging to the NORMAL class, we
sample in such a way to obtain a 50/50 TUMOR/NORMAL split on average per batch.

In [ ]:
import threading

batch_size = 32 # batch size per gradient update (identical to what was used in Liu et al.)

def preprocess_input(x):
    """Scales the image pixel values to be within the range of [-1,1]
    for use with the Inception model.
    """
    return 2*(x/255.0)-1.0

class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def next(self):
        with self.lock:
            return self.it.next()


def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g

@threadsafe_generator
def pathology_training_img_generator(dirpath):
    """A generator that samples from the training dataset and returns
    a tensor of size (batch_size, img_width, img_heigth, number_of_channels).
    
    This generator also implements the sampling scheme to ensure class balance
    """
    files = os.listdir(dirpath)
    X = np.zeros((batch_size, 299, 299, 3)) # container for training image patches, with the number of rows equal to the batch size
    Y = np.ones((batch_size)) # container for ground truth labels
    
    file_types = ['Tumor_Slide_', 'Normal_Slide_'] 
    # We use an 80/20 train/validation split
    num_norm_slide = 128 # first 128 of 160 normal slides used for training
    num_tumor_slide = 88 # first 88 of 110 tumor slide used for training
    
    # The following two variables denote the number of normal or tumor patches randomly sampled from each slide during the preprocessing step
    # to create the training dataset. These values were appended to patch file names so we put a ceiling to the range of possible image 
    # patches during sampling in this function.
    num_norm_patches_per_slide = 10400  
    num_tumor_patches_per_slide = 1040 

    while True:
        for i in xrange(batch_size):
            ftype = np.random.choice(file_types, p=[.75,.25]) # sample from Tumor Slide or Normal Slide with probability p and 1-p respectively
            if ftype == 'Normal_Slide_':
                file_num = "%03d" % np.random.randint(1,num_norm_slide + 1) # randomly sample 1 Tumor slide
                num_patches = "%05d" % np.random.randint(0, num_norm_patches_per_slide, 1) # randomly sample a normal patch
                patch_img = np.load(os.path.join(dirpath, ftype + file_num, 'Normal', 
                                        ftype + file_num + '_Normal_Patch_' + num_patches + '.npy'))
                X[i,:,:,:] =  preprocess_input(patch_img)
                Y[i] = 0
            else:
                file_num = "%03d" % np.random.randint(1,num_tumor_slide + 1)
                num_patches = 0
                patch_types = ['Tumor', 'Normal'] 
                ptype = np.random.choice(patch_types, p=[.67, .33]) # sample from Tumor and Normal patches with probability q and 1-q respectively
                if ptype == 'Tumor':
                    Y[i] = 1
                    num_patches = "%05d" % np.random.randint(0, num_tumor_patches_per_slide, 1) # randomly sample a normal patch
                else:
                    Y[i] = 0
                    num_patches = "%05d" % np.random.randint(0, num_norm_patches_per_slide,1) # randomly sample a tumor patch

                patch_img = np.load(os.path.join(dirpath, ftype + file_num, ptype, 
                                    ftype + file_num + '_' + ptype + '_Patch_' + num_patches + '.npy'))
                X[i,:,:,:] =  preprocess_input(patch_img)

        yield(X,Y)

In the next cell, we randomly sample patches from the remaining 20% split of the training data to used as validation set. We sample in such a way to maintain balance between Normal and Tumor patches.

In [ ]:
from sklearn.utils import shuffle

def get_validation_data(dirpath):
    """The validation data is obtained from the remaining 20% slides. Due to the large computation time if all 
    validation data are used, only 1000 patches (500 normal and 500 tumor) are randomly selected from the
    remaining slides.
    """
    X_val_norm = np.zeros((500, 299, 299, 3))
    Y_val_norm = np.zeros(500)
    X_val_tumor = np.zeros((500, 299, 299, 3))
    Y_val_tumor = np.ones(500)
    
    for i in xrange(500):
        num_normal_slide = "%03d" % np.random.randint(128, 160, 1) # randomly sample a normal slide
        num_normal_patch = "%05d" % np.random.randint(0, 10400, 1) # randomly sample a normal patch
        ptype = 'Normal'
        if num_normal_slide < 110:
            ptype = np.random.choice(['Tumor', 'Normal'])
        num_tumor_slide = "%03d" % np.random.randint(88, 110, 1) # randomly sample a tumor slide
        num_tumor_patch = "%05d" % np.random.randint(0, 1040, 1) # randomly sample a tumor patch
        X_val_norm[i,:,:,:] = preprocess_input(np.load(os.path.join(dirpath, ptype + '_Slide_' + num_normal_slide, 'Normal', 
                                ptype  + '_Slide_' + num_normal_slide + '_Normal_Patch_' + num_normal_patch + '.npy')))
 
        X_val_tumor[i,:,:,:] = preprocess_input(np.load(os.path.join(dirpath, 'Tumor_Slide_' + num_tumor_slide, 'Tumor', 
                                    'Tumor_Slide_' + num_tumor_slide + '_Tumor_Patch_' + num_tumor_patch + '.npy')))
    
    # concatenate the two various design matrix and shuffle the rows
    X_val = np.concatenate((X_val_norm, X_val_tumor), axis=0)
    Y_val = np.concatenate((Y_val_norm, Y_val_tumor), axis=0)
    X_val, Y_val = shuffle(X_val, Y_val, random_state=100)
    return X_val, Y_val

# generate validation data
val_dirpath = '/media/data/home/lizx/split'
X_val, Y_val = get_validation_data(val_dirpath)
            

We note for convolutional models, lower layers represent more primitive shapes, such as lines (edges) and circles, and each successive layers represents higher, more abstract shapes until reaching the top layer which is the most directly related to the particular image at hand. See cell below for example.

In [ ]:
# Plot figure of visualization of CNN layers
from IPython.display import Image
Image(url='https://i.stack.imgur.com/Hl2H6.png')

With this in mind, when finetuning a pretrained model, we would like to use a smaller learning rate at the lower layers (would like to maintain the general properties of these lower filters) and a high learning rate at the top layers to better fit our particular dataset. 

To achieve this mechanism in Keras, we use a similar 3-step training strategy as outlined by Shen, Li. "End-to-end Training for Whole Image Breast Cancer Diagnosis using An All Convolutional Design." arXiv preprint arXiv:1708.09427 (2017).: https://arxiv.org/abs/1708.09427

# 3-Step Training pipeline:
    Step 1: set learning rate to 1e-3 and train the top 2 layers for 5 epochs.
    Step 2: Set learning rate to 1e-4, unfreeze the top 2 inception module and train for 10 epochs,
    Step 3: Set learning rate to 1e-5, unfreeze all layers and train for 50-100 epochs.

In [ ]:
# Step 1 Training: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

from keras.optimizers import RMSprop

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=RMSprop(lr=1e-3, rho=.9, epsilon=1e-08, decay=0.0), 
              loss='binary_crossentropy', metrics=['accuracy'])

from keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger, EarlyStopping

# TensorBoard callback saves loss/accuracy data for visualization with tensorboard
tf_board = TensorBoard(log_dir='tf_board_logs/step1', batch_size=batch_size, write_graph=True)

# ModelCheckpoint callback allows user to save only the best performing model during the training phase
filepath="checkpt_models/step1_weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
model_checkpt = ModelCheckpoint(filepath, monitor='val_acc', 
                                verbose=1, save_best_only=True, mode='max')

# CSVLogger callbacks writes loss/accuracy output per epoch to file for later post-processing and visualization
csv_logger = CSVLogger('csv_logs/step1_train.log')

callbacks_list = [tf_board, model_checkpt, csv_logger]

history = model.fit_generator(pathology_training_img_generator('/media/data/home/lizx/split/'), 
                    epochs=5, steps_per_epoch=10, workers=1, callbacks=callbacks_list, validation_data=(X_val, Y_val))

In [ ]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)


For illustrative purposes, we plot the Inception V3 architecture below. 

The last two inception modules are enclosed by "Concat" operations. This is equivalent to the named 'mixed' in the layer names output by Keras. Therefore if we scrolled up the previous cell, we see that layer 248 ('mixed8') is the last operation before the two last inception modules.

In [ ]:
# Inception V3 architecture
Image(url='https://github.com/tensorflow/models/blob/master/inception/g3doc/inception_v3_architecture.png?raw=true')

In [ ]:
# Step 2 Training: we chose to train the top 2 inception modules, i.e. we will freeze
# the first 248 layers and unfreeze the rest (this time fine-tuning the top 2 inception blocks
# along with the top layers we added initially):
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

# notice the smaller learning rate (lr)
model.compile(optimizer=RMSprop(lr=1e-4, rho=0.9, epsilon=1e-08, decay=0.0), 
              loss='binary_crossentropy', metrics=['accuracy'])


tf_board = TensorBoard(log_dir='tf_board_logs/step2', batch_size=batch_size, write_graph=True)
filepath="checkpt_models/step2_weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
model_checkpt = ModelCheckpoint(filepath, monitor='val_acc', 
                                verbose=1, save_best_only=True, mode='max')
csv_logger = CSVLogger('csv_logs/step2_train.log')
callbacks_list = [tf_board, model_checkpt, csv_logger]

# For the sake of brevity, we set epochs to 10 and steps_per_epoch to 10.
loss = model.fit_generator(pathology_training_img_generator('/media/data/home/lizx/split/'), 
                    epochs=10, steps_per_epoch=10, workers=1, callbacks=callbacks_list, 
                    validation_data=(X_val, Y_val))

In [ ]:
# Step 3 Training: We unfreeze all layers and train the entire model
for layer in model.layers:
    layer.trainable = True

# notice the small learning rate (lr)
model.compile(optimizer=RMSprop(lr=1e-5, rho=0.9, epsilon=1e-08, decay=0.0), 
              loss='binary_crossentropy', metrics=['accuracy'])

tf_board = TensorBoard(log_dir='tf_board_logs/step3', batch_size=batch_size, write_graph=True)
filepath="checkpt_models/step3_weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
model_checkpt = ModelCheckpoint(filepath, monitor='val_acc', 
                                verbose=1, save_best_only=True, mode='max')
csv_logger = CSVLogger('csv_logs/step3_train.log')

# For use when computational resources is limited. Can set criteria to terminate training.
#early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

callbacks_list = [tf_board, model_checkpt, csv_logger]

# For the sake of brevity, we set epochs to 10 and steps_per_epoch to 10. These should be
# longer, say epochs=50 and steps_per_epoch=100
loss = model.fit_generator(pathology_training_img_generator('/media/data/home/lizx/split/'), 
                    epochs=10, steps_per_epoch=10, workers=1, callbacks=callbacks_list, 
                    validation_data=(X_val, Y_val))

# Save learned model for prediction time
#model.save('pathai_model.h5')
del model

Now that the model has been fully trained, we are ready to make inference on new slides.

# Test (Inference) Time

In [ ]:
import openslide
from __future__ import division
from math import floor
import numpy as np
import matplotlib.pylab as plt
import matplotlib.colors as mcol
import os
import keras
from keras.models import load_model

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
# set some memory parameters
config=tf.ConfigProto()
config.gpu_options.visible_device_list = "0"
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

In [ ]:
def run_inference_batch(batch_arr,model):
    """Use the provided model to make prediction on each batch array
    """
    return model.predict(batch_arr)[:,0]

def run_inference_across_slide(slide,model,patch_size=(299,299),stride=(128,128),start=(0,0),size='full',batch_size=1000):
    """Makes inference on a test slide by using a sliding window approach
        patch_size: specifies size of patch images
        stride: the stride size used by sliding window across the original pathology image.
        start: starting pixel location for original pathology image to make inference on
        size: defines the size (in pixel) of the original pathology image to make inference on relative to start location
        batch_size: batch size per inference
    """
    
    # open the slide using OpenSlide
    if type(slide)==str:
        slide_fh=openslide.OpenSlide(slide)
    elif type(slide)==openslide.OpenSlide:
        slide_fh=slide
    else:
        raise TypeError('Unknown slide type: ' + str(type(slide)))
        
    slide_size=slide_fh.level_dimensions[0] # We will make prediction at level 0, the full image scale
    
    # specifies start and end pixel location of window of interest in test slide
    if size=='full':
        end=slide_size
        start=(0,0)
    else:
        end=start[0]+size[0],start[1]+size[1]
    
    # calculates size of heat map due to input values
    heat_map_size=(end[0]-start[0]-patch_size[0])//stride[0]+1,(end[1]-start[1]-patch_size[1])//stride[1]+1
    heat_map_list=[]
    batch_list=[]
    count=0
    
    # calculates total number of batches expected to perform inference on
    total_batch_size = int(len(xrange(start[0],end[0]-patch_size[0]+1,stride[0]))*
                                len(xrange(start[1],end[1]-patch_size[1]+1,stride[1]))/batch_size)
    
    # the actual window sliding inference part
    for x in xrange(start[0],end[0]-patch_size[0]+1,stride[0]):
        for y in xrange(start[1],end[1]-patch_size[1]+1,stride[1]):
            # get image patch from OpenSlide
            image=np.array(slide_fh.read_region((x,y),0,patch_size))[:,:,:3]
            batch_list.append(image)
            if len(batch_list)>=batch_size:
                count+=1
                print(str(count) + " of " + str(total_batch_size) + " batches completed.")
                y_=run_inference_batch(np.stack(batch_list),model)
                heat_map_list.append(y_)
                batch_list=[]
    if len(batch_list)!=0:
        y_=run_inference_batch(np.stack(batch_list),model)
        heat_map_list.append(y_)
        batch_list=[]
    heat_map=np.concatenate(heat_map_list).reshape(heat_map_size[1],heat_map_size[0],order='F').T
    return heat_map
    

In [ ]:
# set some directories
NORMAL_SLIDES_DIR='/media/data/home/ysvang/Camelyon16/Train_Normal'
TUMOR_SLIDES_DIR='/media/data/home/ysvang/Camelyon16/Train_Tumor'
MASK_DIR='/media/data/home/ysvang/Camelyon16/Ground_Truth_Extracted/Mask'
ANNO_DIR='/media/data/home/ysvang/Camelyon16/Ground_Truth_Extracted/XML'

In [ ]:
# load back in the best model here
model=load_model('checkpt_models/step3_weights-improvement-04-0.93.hdf5')

# We will make prediction on a sample slide
test_region_top_left=(68924,131761)
slide_file='/media/data/home/ysvang/Camelyon16/Train_Tumor/Tumor_001.tif'
heat=run_inference_across_slide(slide_file,model,start=test_region_top_left,size=(5000,5000))


In [ ]:
heat.shape

In [ ]:
# Plot the heat map for the slide
plt.imshow(heat.T,cmap='jet',vmin=0,vmax=1)
plt.gcf().set_size_inches((15,12))
plt.colorbar()
plt.show()

This concludes this tutorial.